In [2]:
from tensorflow.keras import layers
from tqdm import tqdm

import GetOldTweets3 as got

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd 
import numpy as np 
import copy
import os

In [3]:
data = pd.read_csv("./data/data_train.csv",encoding='latin-1')
data.head()

,Emotion,Text
0,neutral,There are tons of other paintings that I thin...
1,sadness,"Yet the dog had grown old and less capable , a..."
2,fear,When I get into the tube or the train without ...
3,fear,This last may be a source of considerable disq...
4,anger,She disliked the intimacy he showed towards so...


In [4]:
print(data['Emotion'].unique())

['neutral' 'sadness' 'fear' 'anger' 'joy']


In [5]:
data_ = copy.deepcopy(data)

netral_data = data_[data_.Emotion=='neutral']
sadness_data = data_[data_.Emotion=='sadness']
fear_data = data_[data_.Emotion=='fear']
anger_data = data_[data_.Emotion=='anger']
joy_data = data_[data_.Emotion=='joy']

sub_data = pd.concat([netral_data, sadness_data, fear_data, anger_data, joy_data], axis=0)

# Data explore

In [6]:
data_target=data.groupby('Emotion')

In [7]:
data['Emotion'].value_counts()

sadness    1641
joy        1619
neutral    1616
anger      1566
fear       1492
Name: Emotion, dtype: int64

## Date

At what time do people like to tweet? Is there a clear link between the time of tweeting and the emotion of the content?

In [8]:
data.head()

,Emotion,Text
0,neutral,There are tons of other paintings that I thin...
1,sadness,"Yet the dog had grown old and less capable , a..."
2,fear,When I get into the tube or the train without ...
3,fear,This last may be a source of considerable disq...
4,anger,She disliked the intimacy he showed towards so...


## Words

Words distribution.

In [9]:
from nltk.tokenize import word_tokenize

import warnings
import string
import nltk
import re

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bahk_insung/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Data preprocess

In [10]:
netral_data = data_[data_.Emotion=='neutral']
print(f"netural data shape : {netral_data.shape}")

sadness_data = data_[data_.Emotion=='sadness']
print(f"sadness data shape : {sadness_data.shape}")

fear_data = data_[data_.Emotion=='fear']
print(f"feat data shape : {fear_data.shape}")

anger_data = data_[data_.Emotion=='anger']
print(f"anger data shape : {anger_data.shape}")

joy_data = data_[data_.Emotion=='joy']
print(f"joy data shape : {joy_data.shape}")

emo_data = pd.concat([netral_data, sadness_data, fear_data, anger_data, joy_data], axis=0)
print(data.shape)

emo_data.head()

netural data shape : (1616, 2)
sadness data shape : (1641, 2)
feat data shape : (1492, 2)
anger data shape : (1566, 2)
joy data shape : (1619, 2)
(7934, 2)


,Emotion,Text
0,neutral,There are tons of other paintings that I thin...
8,neutral,Yes .
14,neutral,Perhaps we need to have more babies ! Tina ga...
15,neutral,Why ?
23,neutral,Over there .


---

In [11]:
data = pd.read_csv("./data/training.1600000.processed.noemoticon.csv",encoding='latin-1')

In [12]:
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "TweetText"]
data.columns = DATASET_COLUMNS

In [13]:
data_ = copy.deepcopy(data)

positif_data = data_[data_.target==4].iloc[:80000,:]
netural_data = data_[data_.target==2].iloc[:80000, :]
negative_data = data_[data_.target==0].iloc[:80000,:]

sub_data = pd.concat([positif_data, netural_data, negative_data],axis=0)

# Data explore

In [14]:
data_target=data.groupby('target')

In [15]:
data['target'].value_counts()

4    800000
0    799999
Name: target, dtype: int64

## Date

At what time do people like to tweet? Is there a clear link between the time of tweeting and the emotion of the content?

In [16]:
data.head()

,target,ids,date,flag,user,TweetText
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [17]:
data_ = {'target': data['target'], 'date': data['date']}
df = pd.DataFrame(data_)
df.head()

,target,date
0,0,Mon Apr 06 22:19:49 PDT 2009
1,0,Mon Apr 06 22:19:53 PDT 2009
2,0,Mon Apr 06 22:19:57 PDT 2009
3,0,Mon Apr 06 22:19:57 PDT 2009
4,0,Mon Apr 06 22:20:00 PDT 2009


In [18]:
# lets ensure the 'date' column is in date format
df['date'] = pd.to_datetime(df['date'])

/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [19]:
hour = [ df['date'][i].hour for i in range(len(df['date'])) ]
df['hour'] = hour
df.head()

,target,date,hour
0,0,2009-04-06 22:19:49,22
1,0,2009-04-06 22:19:53,22
2,0,2009-04-06 22:19:57,22
3,0,2009-04-06 22:19:57,22
4,0,2009-04-06 22:20:00,22


In [20]:
hour_data = {'0': [0]*24, '2': [0]*24, '4': [0]*24}
for i in range(len(df['hour'])):
    target = str(df['target'][i])
    hour = int(df['hour'][i])
    hour_data[target][hour] += 1

In [21]:
hour_data = [hour_data['0'], hour_data['2'], hour_data['4']]
# Transpose
hour_data = list(map(list,zip(*hour_data)))

## Words

Words distribution.

In [22]:
newdata = copy.deepcopy(sub_data)
newdata.drop(['ids','date','flag','user'],axis = 1,inplace = True)

In [23]:
from nltk.tokenize import word_tokenize

import warnings
import string
import nltk
import re

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bahk_insung/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Data preprocess

In [24]:
positif_data = data[data.target==4]
print(positif_data.shape)

negative_data = data[data.target==0]
print(negative_data.shape)

deep_data = pd.concat([positif_data,negative_data],axis = 0)
print(data.shape)

deep_data.head()

(800000, 6)
(799999, 6)
(1599999, 6)


,target,ids,date,flag,user,TweetText
799999,4,1467822272,Mon Apr 06 22:22:45 PDT 2009,NO_QUERY,ersle,I LOVE @Health4UandPets u guys r the best!!
800000,4,1467822273,Mon Apr 06 22:22:45 PDT 2009,NO_QUERY,becca210,im meeting up with one of my besties tonight! ...
800001,4,1467822283,Mon Apr 06 22:22:46 PDT 2009,NO_QUERY,Wingman29,"@DaRealSunisaKim Thanks for the Twitter add, S..."
800002,4,1467822287,Mon Apr 06 22:22:46 PDT 2009,NO_QUERY,katarinka,Being sick can be really cheap when it hurts t...
800003,4,1467822293,Mon Apr 06 22:22:46 PDT 2009,NO_QUERY,_EmilyYoung,@LovesBrooklyn2 he has that effect on everyone


In [25]:
data['Clean_TweetText'] = data['TweetText'].str.replace("@", "") 
data['Clean_TweetText'] = data['Clean_TweetText'].str.replace(r"http\S+", "") 
data['Clean_TweetText'] = data['Clean_TweetText'].str.replace("[^a-zA-Z]", " ") 

stopwords=nltk.corpus.stopwords.words('english')

def remove_stopwords(text):
    clean_text=' '.join([word for word in text.split() if word not in stopwords])
    return clean_text

/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_64380/1406119821.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Clean_TweetText'] = data['Clean_TweetText'].str.replace(r"http\S+", "")
/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_64380/1406119821.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Clean_TweetText'] = data['Clean_TweetText'].str.replace("[^a-zA-Z]", " ")


In [26]:
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda text : remove_stopwords(text.lower()))
data.head()

,target,ids,date,flag,user,TweetText,Clean_TweetText
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dived many times ball managed save re...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole body feels itchy like fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behaving mad see
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew,kwesidei whole crew


In [27]:
# Text Tokenization and Normalization
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda x: word_tokenize(x))
data.head()

,target,ids,date,flag,user,TweetText,Clean_TweetText
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,"[upset, update, facebook, texting, might, cry,..."
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,"[kenichan, dived, many, times, ball, managed, ..."
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,"[whole, body, feels, itchy, like, fire]"
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","[nationwideclass, behaving, mad, see]"
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew,"[kwesidei, whole, crew]"


In [28]:
# Now let’s stitch these tokens back together
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda x: ' '.join([w for w in x]))
# Removing small words
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
data.head()

,target,ids,date,flag,user,TweetText,Clean_TweetText
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset update facebook texting might result sch...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dived many times ball managed save re...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole body feels itchy like fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behaving
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew,kwesidei whole crew


# Models

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from imblearn.under_sampling import RandomUnderSampler

from tensorflow.keras import layers
from tensorflow.keras import Model
from keras.utils import plot_model

import tensorflow as tf

In [30]:
count_vectorizer = CountVectorizer(stop_words='english') 
emo_cv = count_vectorizer.fit_transform(emo_data['Text'])
print(emo_cv.shape)

(7934, 9887)


In [31]:
undersampler_ratio = {
    "neutral" : 1000,
    "joy" : 1000,
    "sadness" : 1000,
    "anger" : 1000,
    "fear" : 1000
}

rus = RandomUnderSampler(random_state=42, sampling_strategy=undersampler_ratio)
emo_X, emo_y = rus.fit_resample(emo_cv, emo_data['Emotion'])

print(emo_X.shape)
print(emo_y.shape)

(5000, 9887)
(5000,)


In [32]:
print(emo_data['Emotion'].unique())

onehot_label = list()
for value in emo_y:
    if value == "neutral":
        onehot_label.append([1, 0, 0, 0, 0])

    elif value == "sadness":
        onehot_label.append([0, 1, 0, 0, 0])

    elif value == "fear":
        onehot_label.append([0, 0, 1, 0, 0])

    elif value == "anger":
        onehot_label.append([0, 0, 0, 1, 0])

    elif value == "joy":
        onehot_label.append([0, 0, 0, 0, 1])

    else:
        break

onehot_label = np.array(onehot_label)

['neutral' 'sadness' 'fear' 'anger' 'joy']


In [33]:
emo_X = emo_X.toarray()

emo_X_train, emo_X_test, emo_y_train, emo_y_test = train_test_split(emo_X, onehot_label, test_size=.33, random_state=42)
print(f'''
X_train shape : {emo_X_train.shape}
y_train shape : {emo_y_train.shape}

X_test shape : {emo_X_test.shape}
y_test shape : {emo_y_test.shape}
''')


X_train shape : (3350, 9887)
y_train shape : (3350, 5)

X_test shape : (1650, 9887)
y_test shape : (1650, 5)



# Deepression y onehot

In [34]:
count_vectorizer = CountVectorizer(stop_words='english') 
deep_cv = count_vectorizer.fit_transform(data['Clean_TweetText'])
deep_cv.shape

(1599999, 546404)

In [35]:
undersampler_ratio = {
    0 : 2500,
    4 : 2500
}

rus = RandomUnderSampler(random_state=42, sampling_strategy=undersampler_ratio)
deep_X, deep_y = rus.fit_resample(deep_cv, data['target'])

print(deep_X.shape)
print(deep_y.shape)

(5000, 546404)
(5000,)


In [36]:
deep_y_onehot = list()
for value in deep_y:
    if value == 0:
        deep_y_onehot.append([1, 0])

    elif value == 4:
        deep_y_onehot.append([0, 1])
    
    else:
        break

deep_y_onehot = np.array(deep_y_onehot)
print(deep_y_onehot.shape)

(5000, 2)


In [37]:
deep_X = deep_X.toarray()

deep_X_train, deep_X_test, deep_y_train, deep_y_test = train_test_split(deep_X, deep_y_onehot, test_size=.33, random_state=42)
print(f'''
X_train shape : {deep_X_train.shape}
y_train shape : {deep_y_train.shape}

X_test shape : {deep_X_test.shape}
y_test shape : {deep_y_test.shape}
''')


X_train shape : (3350, 546404)
y_train shape : (3350, 2)

X_test shape : (1650, 546404)
y_test shape : (1650, 2)



In [66]:
emo_input_layer = layers.Input(shape=(1, 9887), name="emotion_model_input")
x1 = layers.Conv1D(2048, 3, padding='same', activation='relu')(emo_input_layer)
x1 = layers.Embedding(2048, 1024)(x1)
x1 = layers.GlobalAveragePooling2D()(x1)
x1 = layers.Reshape((1, 1024))(x1)

x1 = layers.Conv1D(1024, 3, padding='same', activation='relu')(x1)
x1 = layers.Conv1D(1024, 3, padding='same', activation='relu')(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.MaxPool1D(pool_size=(2), strides=2, padding='same')(x1)

x1 = layers.Conv1D(512, 3, padding='same', activation='relu')(x1)
x1 = layers.Conv1D(512, 3, padding='same', activation='relu')(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.MaxPool1D(pool_size=(2), strides=2, padding='same')(x1)
x1 = layers.Dropout(0.5)(x1)

x1 = layers.Conv1D(512, 3, padding='same', activation='relu')(x1)
x1 = layers.Conv1D(512, 3, padding='same', activation='relu')(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.MaxPool1D(pool_size=(2), strides=2, padding='same')(x1)

x1 = layers.Conv1D(256, 3, padding='same', activation='relu')(x1)
x1 = layers.Conv1D(256, 3, padding='same', activation='relu')(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.MaxPool1D(pool_size=(2), strides=2, padding='same')(x1)
temp_x1 = layers.Flatten()(x1)
temp_emo_y = layers.Dense(5, activation='softmax', name="Before_GRU_emo")(temp_x1)

x1 = layers.Reshape((1, 256))(x1)
x1 = layers.GRU(256)(x1)
x1 = layers.Dropout(0.5)(x1)

x1 = layers.Flatten()(x1)
x1 = layers.Dense(50)(x1)
x1 = layers.Dense(30)(x1)
x1 = layers.Dense(15)(x1)
emo_y = layers.Dense(5, activation='softmax', name="final")(x1)
emo_model = Model(inputs=emo_input_layer, outputs=[emo_y, temp_emo_y])

In [67]:
emo_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 emotion_model_input (InputLaye  [(None, 1, 9887)]   0           []                               
 r)                                                                                               
                                                                                                  
 conv1d_38 (Conv1D)             (None, 1, 2048)      60747776    ['emotion_model_input[0][0]']    
                                                                                                  
 embedding_18 (Embedding)       (None, 1, 2048, 102  2097152     ['conv1d_38[0][0]']              
                                4)                                                                
                                                                                            

In [68]:
# deep_input_layer = layers.Input(shape=(1, 546404), name="deep_model_input")
# x = layers.Conv1D(2048, 3, padding='same', activation='relu')(deep_input_layer)

# x = layers.Conv1D(1024, 3, padding='same', activation='relu')(x)
# x = layers.Conv1D(1024, 3, padding='same', activation='relu')(x)
# x = layers.BatchNormalization()(x)
# x = layers.MaxPool1D(pool_size=(2), strides=2, padding='same')(x)

# x = layers.Conv1D(1024, 3, padding='same', activation='relu')(x)
# x = layers.Conv1D(1024, 3, padding='same', activation='relu')(x)
# x = layers.BatchNormalization()(x)
# x = layers.MaxPool1D(pool_size=(2), strides=2, padding='same')(x)

# x = layers.Conv1D(512, 3, padding='same', activation='relu')(x)
# x = layers.Conv1D(512, 3, padding='same', activation='relu')(x)
# x = layers.BatchNormalization()(x)
# x = layers.MaxPool1D(pool_size=(2), strides=2, padding='same')(x)

# x = layers.Conv1D(512, 3, padding='same', activation='relu')(x)
# x = layers.Conv1D(512, 3, padding='same', activation='relu')(x)
# x = layers.BatchNormalization()(x)
# x = layers.MaxPool1D(pool_size=(2), strides=2, padding='same')(x)

# x = layers.Conv1D(256, 3, padding='same', activation='relu')(x)
# x = layers.Conv1D(256, 3, padding='same', activation='relu')(x)
# x = layers.BatchNormalization()(x)
# x = layers.MaxPool1D(pool_size=(2), strides=2, padding='same')(x)

# x = layers.Conv1D(256, 3, padding='same', activation='relu')(x)
# x = layers.Conv1D(256, 3, padding='same', activation='relu')(x)
# x = layers.BatchNormalization()(x)
# x = layers.MaxPool1D(pool_size=(2), strides=2, padding='same')(x)
# temp_deep_y = layers.Dense(2, activation='softmax', name="Before_GRU_deep")(x)

# x = layers.Reshape((1, 256))(x)
# x = layers.GRU(256)(x)

# x = layers.Reshape((4, 64))(x)
# x = layers.GRU(64)(x)

# x = layers.Dense(50)(x)
# x = layers.Dense(30)(x)
# x = layers.Dense(15)(x)
# deep_y = layers.Dense(2, activation='softmax')(x)
# deep_model = Model(inputs=deep_input_layer, outputs=[deep_y, temp_deep_y])

In [69]:
# deep_model.summary()

In [70]:
# combined_model = Model(inputs=[deep_input_layer, emo_input_layer], outputs=[temp_deep_y, deep_y, temp_emo_y, emo_y])
# combined_model.summary()

In [71]:
optim = tf.keras.optimizers.Adam(
    lr=0.003
)

emo_model.compile(
    loss='categorical_crossentropy',
    optimizer=optim,
    metrics=['accuracy']
)

plot_model(emo_model, show_shapes=True, to_file='model_visualization.png')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [72]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

outDir = './cheakpoint/' 
model_names = outDir + 'weights-{val_final_accuracy:.4f}.h5'
def get_callbacks(patience = 50):
    model_checkpoint = ModelCheckpoint(model_names, monitor='val_final_accuracy', verbose=1, save_best_only=True, period = 1)
    callbacks = [model_checkpoint]

    return callbacks

In [73]:
emo_X_train = emo_X_train.reshape(-1, 1, 9887)
emo_X_test = emo_X_test.reshape(-1, 1, 9887)

In [74]:
emo_X_test.shape
emo_y_test.shape

(1650, 5)

In [75]:
callbacks = get_callbacks()

history = emo_model.fit(
    emo_X_train, emo_y_train,
    shuffle=True,
    batch_size=128,
    epochs=100,
    validation_data=(emo_X_test, emo_y_test),
    callbacks=[callbacks]
)

Epoch 1/100


2022-12-29 21:01:39.769053: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-29 21:01:40.614371: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-29 21:01:41.456652: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


27/27 [==============================] - ETA: 0s - loss: 3.2232 - final_loss: 1.6126 - Before_GRU_emo_loss: 1.6106 - final_accuracy: 0.1830 - Before_GRU_emo_accuracy: 0.1967

2022-12-29 21:02:14.433988: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-29 21:02:14.783993: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_final_accuracy improved from -inf to 0.19818, saving model to ./cheakpoint/weights-0.1982.h5
27/27 [==============================] - 51s 2s/step - loss: 3.2232 - final_loss: 1.6126 - Before_GRU_emo_loss: 1.6106 - final_accuracy: 0.1830 - Before_GRU_emo_accuracy: 0.1967 - val_loss: 3.2206 - val_final_loss: 1.6108 - val_Before_GRU_emo_loss: 1.6098 - val_final_accuracy: 0.1982 - val_Before_GRU_emo_accuracy: 0.1909
Epoch 2/100
27/27 [==============================] - ETA: 0s - loss: 3.2901 - final_loss: 1.6273 - Before_GRU_emo_loss: 1.6628 - final_accuracy: 0.2009 - Before_GRU_emo_accuracy: 0.2015
Epoch 2: val_final_accuracy improved from 0.19818 to 0.20727, saving model to ./cheakpoint/weights-0.2073.h5
27/27 [==============================] - 80s 3s/step - loss: 3.2901 - final_loss: 1.6273 - Before_GRU_emo_loss: 1.6628 - final_accuracy: 0.2009 - Before_GRU_emo_accuracy: 0.2015 - val_loss: 3.2194 - val_final_loss: 1.6093 - val_Before_GRU_emo_loss: 1.6101 - val_final_accurac

In [ ]:
def plot_model__hist(hist):
    path = './cheakpoint/lefms/' # loss, accuracy 그래프 저장할 path

    # loss 추이 그래프로 그려서 저장
    plt.figure(figsize=(6,6))
    plt.style.use("ggplot")
    plt.plot(hist.history['loss'], color='b', label="Training loss")
    plt.plot(hist.history['val_loss'], color='r', label="Validation loss")
    plt.legend()
    plt.show()

    # accuracy 추이 그래프로 그려서 저장
    plt.figure(figsize=(6,6))
    plt.style.use("ggplot")
    plt.plot(hist.history['accuracy'], color='b', label="Training accuracy")
    plt.plot(hist.history['val_accuracy'], color='r',label="Validation accuracy")
    plt.legend(loc = "lower right")
    plt.show()

In [ ]:
plot_model__hist(history)
loss,acc = emo_model.evaluate(emo_X_test, emo_y_test, verbose=2)
print("multi_model의 정확도: {:5.2f}%".format(100*acc))
print("multi_model의 Loss: {}".format(loss))

# Model Test (Confusion Matrix)

In [ ]:
from tesnorflow import keras

model_path = './checkpoint/'
model_path = model_path + sorted(os.listdir(model_path))[-1]

recon_model = keras.models.load_model(model_path)
print(model_path)